In [1]:
import requests, os, pandas as pd
from dotenv import load_dotenv
load_dotenv()
from langchain_chroma import Chroma

In [2]:
headers = {
    "authorization": os.getenv("myserver_api_key")
}

embed_url = os.getenv("myserver_url")+":8000/embed"
rerank_url = os.getenv("myserver_url")+":8000/rerank"

In [3]:
class CustomEmbeddings:
    def embed_documents(self, texts):
        payload = {
            "texts": texts,
            "is_query": False
        }
        return requests.post(embed_url, json=payload, headers=headers).json()["embeddings"]
    
    def embed_query(self, text):
        payload = {
            "texts": [text],
            "is_query": True
        }
        # return requests.post(embed_url, json=payload, headers=headers).json()["embeddings"][0]
        return requests.post(embed_url, json=payload, headers=headers)

In [4]:
embeddings = CustomEmbeddings()

In [5]:
DATA_PATH = "../data/parent_split_docs_20250909.csv"
DB_PATH = "../chroma_db"

In [6]:
vector_store = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embeddings,
)

In [7]:
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 10, "fetch_k": 50}
)

In [8]:
res = embeddings.embed_query("CoreFlow 휴가 규정")

In [9]:
res

<Response [500]>

In [ ]:
docs_list = retriever.invoke("CoreFlow 휴가 규정")